In [ ]:
!pip install boto3

     |████████████████████████████████| 132 kB 20.8 MB/s 
     |████████████████████████████████| 8.6 MB 58.4 MB/s 
     |████████████████████████████████| 79 kB 9.8 MB/s 
     |████████████████████████████████| 138 kB 32.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip freeze

absl-py==1.0.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.11.4
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==4.1.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.4
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.10
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2021.10.8
cffi==1.15.0
cftime==1.5.2
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda111==9.4.0
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.28
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill=

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import pandas as pd
import numpy as np
import boto3

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Creating the necessary folder structure to download s3 files to

In [ ]:
import os 
print(os.getcwd())
%mkdir mask_data
%mkdir mask_data/train
%mkdir mask_data/train/mask
%mkdir mask_data/train/no_mask
%mkdir mask_data/test/mask
%mkdir mask_data/test/no_mask
%mkdir mask_data/test

/content
mkdir: cannot create directory ‘mask_data/test/mask’: No such file or directory
mkdir: cannot create directory ‘mask_data/test/no_mask’: No such file or directory


### Download S3 files into local storage ###

In [ ]:
#Import AWS Credentials that you will need
aws_access_key = 'AKIAXN'
aws_secret_key = 'KFKjtNo8'

In [ ]:
#Setting the train mask directory
mask_train_path = "./mask_data/train/mask"
no_mask_train_path = "./mask_data/train/no_mask"

In [ ]:
import boto3  # pip install boto3

# Connecting to S3 bucket
s3 = boto3.resource("s3", aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key)

#Downloading the Training Images 
bucket = s3.Bucket('511004593648-msds436')
for obj in bucket.objects.filter(Prefix = 'new_with_mask'):
    print(obj.key)
    file_name = obj.key.split("/")[-1]
    file_path = os.path.join(mask_train_path, file_name)
    print("Downloading to {}".format(file_path))
    bucket.download_file(obj.key, file_path)

In [ ]:
#Downloading the Training Images 
bucket = s3.Bucket('511004593648-msds436')
for obj in bucket.objects.filter(Prefix = 'new_without_mask'):
    print(obj.key)
    file_name = obj.key.split("/")[-1]
    file_path = os.path.join(no_mask_train_path, file_name)
    print("Downloading to {}".format(file_path))
    bucket.download_file(obj.key, file_path)

Need to create the following file structure:
- content
  - mask_data
    - test
      - mask
      - no_mask
    - train
      - mask
      - no mask

In [ ]:
train_file_directory = "./mask_data/train/"
test_file_directory = "./mask_data/test/"

In [ ]:
%rm -r ./mask_data/train/.ipynb_checkpoints
%rm -r ./mask_data/test/.ipynb_checkpoints


rm: cannot remove './mask_data/train/.ipynb_checkpoints': No such file or directory


In [ ]:
#Create the train dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    "./mask_data/train/", class_names=['mask', 'no_mask'], label_mode='int', color_mode='rgb', batch_size=1, image_size=(256,
    256), shuffle=True, seed=None, validation_split=None, subset=None,
    interpolation='bilinear', follow_links=False,
    crop_to_aspect_ratio=False
)

Found 20000 files belonging to 2 classes.


In [ ]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
print(type(train_ds))

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


AttributeError: ignored

In [ ]:
#Create the test dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_file_directory, labels='inferred', label_mode='int',
    class_names=['mask', 'no_mask'], color_mode='rgb', batch_size=1, image_size=(256,
    256), shuffle=True, seed=None, validation_split=None, subset=None,
    interpolation='bilinear', follow_links=False,
    crop_to_aspect_ratio=False
)

Found 20 files belonging to 2 classes.


In [ ]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False ## Not trainable weights

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

TypeError: ignored

Trial with an example

In [ ]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
prediction_layer = layers.Dense(1, activation='sigmoid')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    prediction_layer
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

model.fit(train_ds, epochs=1, batch_size=1000)


20000/20000 [==============================] - 911s 45ms/step - loss: 0.0098 - accuracy: 0.9997


In [ ]:
pred = model.predict(test_ds)
print(pred.shape)

(20, 1)


In [ ]:
pd.DataFrame({'predicted': pred.reshape(20,), 'actual':np.concatenate([y for x, y in test_ds], axis=0)})

,predicted,actual
0,1.0,1
1,0.0,1
2,0.0,0
3,1.0,0
4,0.0,1
5,0.0,0
6,1.0,1
7,1.0,1
8,1.0,1
9,1.0,0


In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/mask_model')

INFO:tensorflow:Assets written to: saved_model/mask_model/assets


In [ ]:
(test_train_ds, test_train_labels), (test_test_ds, test_test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]


Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In [ ]:
print(type(test_train_labels))
print(test_train_ds[0].shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(442, 1024, 3)


Save to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cv2.imwrite('/content/drive/My Drive/Delete/res.png' , rgb_img)

In [ ]:
!zip -r mask_model.zip saved_model/

  adding: saved_model/ (stored 0%)
  adding: saved_model/mask_model/ (stored 0%)
  adding: saved_model/mask_model/variables/ (stored 0%)
  adding: saved_model/mask_model/variables/variables.data-00000-of-00001 (deflated 21%)
  adding: saved_model/mask_model/variables/variables.index (deflated 65%)
  adding: saved_model/mask_model/keras_metadata.pb (deflated 96%)
  adding: saved_model/mask_model/assets/ (stored 0%)
  adding: saved_model/mask_model/saved_model.pb (deflated 90%)


In [ ]:
cv2.imwrite('/content/drive/My Drive/Delete/mask_model.zip' , )

In [ ]:
!cp "/content/mask_model.zip" "/content/drive/My Drive/mask_model.zip"